# Notebook to overplot MIRI photometry on existing PROSPECTOR fits

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
import prospect.io.read_results as reader
import pickle as pkl

from astropy.io import fits
from astropy import units as u
from prospect.io.read_results import results_from
from astropy.table import Table
from miri_utils.prospector_utils import plot_reconstruction_with_miri


Let's load the data from some example output

In [ ]:
catalogue =  '/Users/benjamincollins/University/master/Red_Cardinal/photometry/catalogues/cat_targets.fits'

with fits.open(catalogue) as catalog_hdul:
        cat_data = catalog_hdul[1].data
        ids = cat_data['id']
        ra = cat_data['ra']
        dec = cat_data['dec']
        zred = cat_data['z_best']

# Combine IDs and redshifts into an array of tuples
output_array = np.array(list(zip(ids, zred)), dtype=[('galid', int), ('zred', '<f8')])

# Save to text file
np.savetxt('/Users/benjamincollins/University/master/Red_Cardinal/photometry/catalogues/redshifts.txt',
           output_array,
           fmt='%d %.8f',
           header='galid zred')

# Check output files
res, obs, mod = results_from('/Users/benjamincollins/University/master/Red_Cardinal/prospector/outputs/output_7102_allfal_v1.4phot_0.3sigma_solar_marg_1733274947_mcmc.h5')
print(res.keys())
print(res['theta_labels'])       # Parameter names
print(res['chain'].shape)        # Shape of the posterior samples
print(res['bestfit'].keys())     # Best-fit photometry, spectrum, chi2, etc.


In [ ]:
# Check output files
res, obs, mod = results_from('/Users/benjamincollins/University/master/Red_Cardinal/prospector/outputs/output_7102_allfal_v1.4phot_0.3sigma_solar_marg_1733274947_mcmc.h5')
print(res.keys())
print(res['theta_labels'])       # Parameter names
print(res['chain'].shape)        # Shape of the posterior samples
print(res['bestfit'].keys())     # Best-fit photometry, spectrum, chi2, etc.


Inspect Amir's photometry table

In [ ]:
# Load the bluejay catalogue
bluejay_phot = fits.open('/Users/benjamincollins/University/master/Red_Cardinal/photometry/catalogues/bluejay_phot_cat_v1.4.fits')[1].data
#bluejay_phot = fits.open('/Users/benjamincollins/University/master/Red_Cardinal/photometry/catalogues/cat_targets.fits')[1].data
bluejay_phot = Table(bluejay_phot)
print(bluejay_phot.colnames)  # Print column names to inspect the table structure

Example usage:

In [ ]:
# Example: plot results for galaxy with ID 12345 using specific h5 file

table_path = '/Users/benjamincollins/University/Master/Red_Cardinal/photometry/phot_tables/Photometry_Table_MIRI_v6.fits'

table = Table.read(table_path, format='fits')
galaxy_ids = np.asarray([str(gid) for gid in table['ID']])

output_dir = '/Users/benjamincollins/University/master/Red_Cardinal/prospector/fits_plus_miri/'

x=0
while x<5:
    for objid in galaxy_ids:
        plot_reconstruction_with_miri(int(objid), output_dir)
        x += 1
        
#plot_reconstruction_with_miri(7922, output_dir=output_dir)
#plot_reconstruction_with_miri(9871, output_dir=output_dir)
#plot_reconstruction_with_miri(12717, output_dir=output_dir)

In [ ]:
# Directory containing your PROSPECTOR outputs
output_dir = "/Users/benjamincollins/University/master/Red_Cardinal/prospector_outputs/"

# Example: plot a single object
objid = 16874 #19681  # Replace with your object ID
objid = 16615

# Find the output file for this object
import os
output_files = [f for f in os.listdir(output_dir) if f.endswith('.h5')]
matching_files = [f for f in output_files if f'_{objid}_' in f]

if matching_files:
    output_file = os.path.join(output_dir, matching_files[0])
    phot_table = '/Users/benjamincollins/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI_v4.fits'
    plot_existing_fit_with_miri(objid, phot_table, output_file)
else:
    print(f"No output file found for object {objid}")

# Or plot multiple objects:
# objid_list = [12345, 67890, 11111]  # Replace with your object IDs
# plot_multiple_objects(objid_list, output_dir)

In [ ]:
npy_file = '/Users/benjamincollins/University/master/Red_Cardinal/prospector/obs/obs_12717.npy'
obs = np.load(npy_file, allow_pickle=True).item()

param_file = '/Users/benjamincollins/University/master/Red_Cardinal/prospector/params/params_MAP_12717.pkl'
params = pkl.load(open(param_file, 'rb'))
#print(params.keys())

param_file = '/Users/benjamincollins/University/master/Red_Cardinal/prospector/spec_calib/spec_calibrated_12717.pkl'
data = pkl.load(open(param_file, 'rb'))
print(data['emi_off']['MAP'].keys())

obs_file = f'/Users/benjamincollins/University/master/Red_Cardinal/prospector/obs/obs_12717.npy'
obs = np.load(obs_file, allow_pickle=True).item()
print(obs.keys())
print(len(obs['mask']))
print(len(obs['spectrum']))
print(len(obs['spectrum'][obs['mask']]))

plt.plot(obs['wavelength'][obs['mask']], obs['spectrum'][obs['mask']], label="Observed spectrum")
#plt.plot(data['wave_obs'], data['MAP']['wave_obs'], label="MAP spectrum")
#plt.plot(data['wave_obs'], data['nodust'], label="No dust", linestyle="--")
plt.plot(data['wave_obs'], data['emi_off']['MAP']['wave_obs'], label="No emission lines", linestyle="-")
plt.loglog()
plt.xlabel("Observed wavelength [Å]")
plt.ylabel("Flux [maggies or arbitrary units]")
plt.legend()
plt.title("Calibrated Model Spectrum Variants")
plt.show()